<a href="https://colab.research.google.com/github/hila-chefer/Transformer-Explainability/blob/main/BERT_explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !git clone https://github.com/hila-chefer/Transformer-Explainability.git

import os
# os.chdir(f'./Transformer-Explainability')

# !pip install -r requirements.txt
!pip install captum

In [2]:
!pip install captum==0.6.0
# !pip install matplotlib==3.3.2

In [3]:
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from transformers import AutoTokenizer

from captum.attr import visualization
import torch

/home/dulanga/.conda/envs/isuranga-explainability/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2").to("cuda")
model.eval()
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")
# initialize the explanations generator
explanations = Generator(model)

classifications = ["NEGATIVE", "POSITIVE"]


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at textattack/bert-base-uncased-SST-2 and are newly initialized: ['bert.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#Positive sentiment example

In [5]:
# encode a sentence
text_batch = ["This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

/home/dulanga/.conda/envs/isuranga-explainability/lib/python3.10/site-packages/transformers/modeling_utils.py:1621: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


[('[CLS]', 0.0), ('this', 0.4277416169643402), ('movie', 0.3126758635044098), ('was', 0.2679917514324188), ('the', 0.3673456013202667), ('best', 0.6282141208648682), ('movie', 0.28917035460472107), ('i', 0.19019818305969238), ('have', 0.10187963396310806), ('ever', 0.145744189620018), ('seen', 0.18924416601657867), ('!', 0.6002076864242554), ('some', 0.007671934552490711), ('scenes', 0.035823483020067215), ('were', 0.019184978678822517), ('ridiculous', 0.022461874410510063), (',', 0.0), ('but', 0.42851656675338745), ('acting', 0.44285157322883606), ('was', 0.49728918075561523), ('great', 1.0), ('.', 0.015530857257544994), ('[SEP]', 0.08978144079446793)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


#Negative sentiment example

In [6]:
# encode a sentence
text_batch = ["I really didn't like this movie. Some of the actors were good, but overall the movie was boring."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                1,
                                1,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', -0.0), ('i', -0.19129253923892975), ('really', -0.1890297383069992), ('didn', -0.2899225056171417), ("'", -0.006506895646452904), ('t', -0.36849480867385864), ('like', -0.15265028178691864), ('this', -0.18944473564624786), ('movie', -0.04036051407456398), ('.', -0.019485048949718475), ('some', -0.022970611229538918), ('of', -0.0), ('the', -0.02299061417579651), ('actors', -0.09536828100681305), ('were', -0.01349655631929636), ('good', -0.032404981553554535), (',', -0.004253493621945381), ('but', -0.058687880635261536), ('overall', -0.3361990749835968), ('the', -0.21855174005031586), ('movie', -0.05484887212514877), ('was', -0.625758171081543), ('boring', -1.0), ('.', -0.031012842431664467), ('[SEP]', -0.052513591945171356)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (1.00),1,1.00,"[CLS] i really didn ' t like this movie . some of the actors were good , but overall the movie was boring . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (1.00),1,1.00,"[CLS] i really didn ' t like this movie . some of the actors were good , but overall the movie was boring . [SEP]"


# Choosing class for visualization example

In [14]:
# encode a sentence
text_batch = ["I hate that I love you."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
target_class = 0
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=11, index=target_class)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)

# get class name
class_name = classifications[target_class]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', -0.0), ('i', -0.19790242612361908), ('hate', -1.0), ('that', -0.40287283062934875), ('i', -0.12505637109279633), ('love', -0.1307140290737152), ('you', -0.05467141419649124), ('.', -6.108225989009952e-06), ('[SEP]', -0.0)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.91),1,1.00,[CLS] i hate that i love you . [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.91),1,1.00,[CLS] i hate that i love you . [SEP]


In [15]:
# encode a sentence
text_batch = ["I hate that I love you."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
target_class = 1
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=11, index=target_class)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)

# get class name
class_name = classifications[target_class]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('i', 0.2725590765476227), ('hate', 0.17270179092884064), ('that', 0.23211266100406647), ('i', 0.17642731964588165), ('love', 1.0), ('you', 0.2465524971485138), ('.', 0.0), ('[SEP]', 0.00015733683540020138)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.91),1,1.00,[CLS] i hate that i love you . [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.91),1,1.00,[CLS] i hate that i love you . [SEP]
